In [171]:
import pandas as pd
import os

In [203]:
allgsdf = pd.read_csv("../../resources/20180622processedGoldStandardTopics.tsv.gz", delimiter="\t")
#allgsdf = pd.read_csv("../../resources/20190111processedGoldStandardPub2018.tsv.gz", delimiter="\t")
gsdf = allgsdf.drop(["title", "abstract", "major_mesh", "minor_mesh"], axis=1)

# Read all results

In [173]:
def calculateRelFoundCounts(dir):
    # Read all results in the given result directory
    resultfiles = sorted(os.listdir(dir))
    # Read the result files as DataFrames into a map
    resultdfmap = []
    for f in resultfiles:
        resultdfmap.append(pd.read_csv(dir+f, delimiter="\t", names=["topic", "Q0", "docid", "rank", "score", "run"]))
    # Create a DataFrame multiindexed with the file name (because those are the keys of the DF maps)
    resultmultidxdf = pd.concat(resultdfmap)
    resultmultidxdf.set_index(["run"], inplace=True)
    
    # Create a duplication of the relevant GS document to match the results
    gsreldocs = gsdf.query("relevance_score > 0")[["trec_topic_number", "trec_doc_id"]]
    l = []
    for experiment in set(resultmultidxdf.index):
        idx = pd.Index([experiment]*len(gsreldocs), name="run")
        gscopy = gsreldocs.copy()
        gscopy.index = idx
        l.append(gscopy)
    gsdfs = pd.concat(l)
    
    # Merge the duplicated GS with the results
    # With a `left` join, thus eliminating all irrelevant documents.
    relmerge = pd.merge(gsdfs, resultmultidxdf, how="left", left_on=["run", "trec_topic_number", "trec_doc_id"], right_on=["run", "topic", "docid"])
    relmerge.set_index("trec_topic_number", append="True", inplace=True)
    
    # Count the number of found documents per run and topic    
    countsruntopic = relmerge.groupby(["run", "trec_topic_number"]).count()
    countsruntopic = countsruntopic.drop(["Q0", "docid", "rank", "score"],axis=1)
    countsruntopic.columns = ["relgs", "relfound"]
    
    # return the left-merged data and the counts
    return relmerge,countsruntopic

In [174]:
def prepareStats(path):
    """
    Reads a single stats CSV file, excludes the 'all' row and converts the topic numbers to ints.
    Then sets the Topic columns as the new index.
    Returns a DataFrame indexed by the non-'all' topics.
    """
    df = pd.read_csv(path).query("Topic != 'all'")
    df["Topic"] = df["Topic"].astype(int)
    df.sort_values(by="Topic", inplace=True)
    df = df.set_index("Topic")
    return df

In [194]:
measures = ["ndcg","infNDCG", "P_10"]

def getMeanStatsPerRun(statspath):
    """
    Reads a directory of stat CSV files. Concatenates all the DataFrames and calculates the means for all
    score measurements of the topics per run, effectively returning the 'all' row for each run. Note,
    however, that really just the mean over the measures is given which should be the 'all' value but the actual
    'all' value is not used here.
    Returns only those measures defined in the 'measures' list at the beginning of this cell.
    """
    statfiles = sorted(list(filter(lambda f: f.endswith(".csv") and not "gspm" in f, os.listdir(statspath))))
    runstatsmap = {}
    for stat in statfiles:
        df = prepareStats(statspath+stat)
        run = stat.replace("OFFICIAL_", "").replace(".csv", "")
        runstatsmap[run] = df
    allstats = pd.concat(runstatsmap)
    allstats.index.names = ["run", "Topic"]
    allstats = allstats[measures]
    meanstats = allstats.mean(level="run")
    return meanstats

In [195]:
def getRelFoundWithMeanRank(resultspath, statspath):
    gsleftmerged,counts = calculateRelFoundCounts(resultspath)
    meanstats = getMeanStatsPerRun(statspath)
    counts   = counts.sum(level="run").sort_values("relfound")
    merge     = pd.merge(counts, meanstats, on="run")
    meanranks = gsleftmerged["rank"].dropna().mean(level="run")
    stdranks  = gsleftmerged["rank"].dropna().std(level="run")
    merge["meanrank"] = meanranks
    merge["stdrank"] = stdranks
    return merge.sort_values("relfound")

In [204]:
getRelFoundWithMeanRank("../../results/", "../../stats_pmclass/")

,relgs,relfound,ndcg,infNDCG,P_10,meanrank,stdrank
run,,,,,,,
hpipubnone_replique_custompm_must-pmclass2017.keyword,3875,2983,0.589833,0.517230,0.640000,644.597385,1093.384068
hpipubnone_replique_custompm_must-pmclass2017lstm.keyword,3875,2983,0.589877,0.516867,0.640000,644.562856,1093.346062
hpipubnone_replique_custompm_must-pmclass2017lstmatt.keyword,3875,2983,0.589983,0.515740,0.640000,644.693262,1093.290460
hpipubnone_replique_custompm_must-pmclass2017lstmgru.keyword,3875,2983,0.589920,0.517147,0.640000,644.736842,1093.515730
hpipubnone_replique_custompm_must-pmclass2018.keyword,3875,2983,0.589853,0.517357,0.640000,644.633590,1093.587813
hpipubnone_replique_custompm_must-pmclass2018lstm.keyword,3875,2983,0.589883,0.516060,0.640000,644.601408,1093.323892
hpipubnone_replique_custompm_must-pmclass2018lstmat.keyword,3875,2983,0.590050,0.516117,0.640000,644.617834,1093.336858
hpipubnone_replique_custompm_must-pmclass2018lstmgru.keyword,3875,2983,0.589820,0.516683,0.640000,644.732819,1093.556256
hpipubnone_replique_custompm_should-pmclass2017.keyword,3875,3623,0.661930,0.545333,0.656667,874.651394,1411.947447
